In [18]:
import pandas as pd
import glob
import os
# import tiktoken

In [15]:
BASE_DIR = 'data/karpathy_llama2.c/'

In [22]:
def combine_parquet_to_jsonl(base_dir, jsonl_file_path):
    # Get a list of all Parquet files in the base directory
    parquet_files = glob.glob(os.path.join(base_dir, '*.parquet'))
    
    # Read each Parquet file into a DataFrame and collect all DataFrames into a list
    dataframes = [pd.read_parquet(file) for file in parquet_files]
    
    # Concatenate all DataFrames into a single DataFrame
    combined_df = pd.concat(dataframes, ignore_index=True)
    
    # Convert to JSONL and save
    combined_df.to_json(jsonl_file_path, orient='records', lines=True)

In [23]:
# Usage
jsonl_file_path = f'{BASE_DIR}/jsonl/llama2.jsonl'
combine_parquet_to_jsonl(BASE_DIR, jsonl_file_path)

In [ ]:
# Load the parquet file
df = pd.read_parquet('data/karpathy_llama2.c/karpathy_llama2.c_commit_data_0.parquet')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402 entries, 0 to 401
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype                               
---  ------                 --------------  -----                               
 0   owner                  402 non-null    string                              
 1   repo_name              402 non-null    string                              
 2   commit_date            402 non-null    datetime64[us, pytz.FixedOffset(60)]
 3   commit_id              402 non-null    string                              
 4   commit_message         402 non-null    string                              
 5   file_path              402 non-null    string                              
 6   previous_commit_id     402 non-null    string                              
 7   previous_file_content  354 non-null    string                              
 8   cur_file_content       376 non-null    string                              
 9  

In [27]:
print('Number of unique commits stored (others excluded for not being code commits):', df.commit_id.nunique())

Number of unique commits stored (others excluded for not being code commits): 273


In [28]:
from pyserini.index.lucene import IndexBuilder

ImportError: cannot import name 'IndexBuilder' from 'pyserini.index' (/opt/homebrew/Caskroom/miniconda/base/envs/ds/lib/python3.9/site-packages/pyserini/index/__init__.py)

In [ ]:
class CodeRepoCollection(Collection):

    def __init__(self, data_frame):
        self.data_frame = data_frame
        self.cursor = 0
        self.max_cursor = len(data_frame)

    def reset(self):
        self.cursor = 0

    def create_filesegment(self, seg_path):
        return CodeRepoFileSegment(self.data_frame[self.cursor:self.cursor + 10000])

    def next_segment(self):
        if self.cursor >= self.max_cursor:
            return None
        segment = self.create_filesegment(self.data_frame[self.cursor:self.cursor + 10000])
        self.cursor += 10000
        return segment

class CodeRepoFileSegment(FileSegment):

    def __init__(self, sub_frame):
        self.sub_frame = sub_frame.reset_index(drop=True)
        self.cursor = 0

    def next(self) -> Document:
        if self.cursor >= len(self.sub_frame):
            return None

        row = self.sub_frame.iloc[self.cursor]
        doc = Document(row['commit_id'], row['tokenized_cur_file_content'])
        doc.metadata = {
            'owner': row['owner'],
            'repo_name': row['repo_name'],
            'file_path': row['file_path']
        }
        self.cursor += 1
        return doc

# Index the tokenized code content
index_args = {
    'collection': CodeRepoCollection(data),
    'generator': 'DefaultLuceneDocumentGenerator',
    'threads': 1,
    'index': 'code_bm25_index',
    'storePositions': True,
    'storeDocvectors': True,
    'storeRaw': True,
    'optimize': True,
}

IndexCollection(index_args)